## Installing NLTK Library for bangla language processing

In [1]:
!pip install bnlp_toolkit

In [2]:
import pandas as pd
import random
import re,nltk,json, pickle
from bnlp import NLTKTokenizer

## Data reading for preprocess
### dataset link: https://www.kaggle.com/furcifer/bangla-newspaper-dataset

In [78]:
df = pd.read_json('../input/bangla-newspaper-dataset/data_v2/data_v2.json')

In [80]:
df['id'] = pd.Series([int(i) for i in range(0,len(df))])
df.head(2)

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content,id
0,গাজীপুর প্রতিনিধি,bangladesh,বাংলাদেশ,"০৪ জুলাই ২০১৩, ২৩:২৬","০৪ জুলাই ২০১৩, ২৩:২৭",[গাজীপুর],0,"কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ, বিক...",http://www.prothom-alo.com/bangladesh/article/...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,0
1,অনলাইন ডেস্ক,sports,খেলা,"০৪ জুলাই ২০১৩, ২৩:০৯","০৪ জুলাই ২০১৩, ২৩:১১",[টেনিস],0,সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি,http://www.prothom-alo.com/sports/article/19028,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...,1


In [82]:
df.head(1)

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content,id
0,গাজীপুর প্রতিনিধি,bangladesh,বাংলাদেশ,"০৪ জুলাই ২০১৩, ২৩:২৬","০৪ জুলাই ২০১৩, ২৩:২৭",[গাজীপুর],0,"কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ, বিক...",http://www.prothom-alo.com/bangladesh/article/...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,0


## News headline and content text processing

In [83]:
def cleaning_headline(title):
    # remove brackets and contents inside this
    content = re.sub("[\(\[].*?[\)\]]", "", title)
    # removing unnecessary punctuation
    content = re.sub('[^\u0980-\u09FF]',' ',str(content))
    k = content.split(" ")
    if len(k)<=3:
        content=""
    return content
    
    
def cleaning_documents(content):
    # remove brackets and contents inside this
    content = re.sub("[\(\[].*?[\)\]]", "", content)
    bnltk = NLTKTokenizer()
    
    k = content.split(" ")
    if len(k)<=10:
        content=""
        return content

    sentence_tokens = bnltk.sentence_tokenize(content)
    sentence_tokens = [item for item in sentence_tokens if len(item)>4]

    if len(sentence_tokens)<20:
        n_split = 3
    elif len(sentence_tokens)>20 and len(sentence_tokens)<150:
        n_split = 5
    elif len(sentence_tokens)>150 and len(sentence_tokens)<500:
        n_split = 10
    elif len(sentence_tokens)>500:
        n_split = 20
    else:
        n_split = 1

    text=""
#     print(n_split)
    cnt=0
    for i in range(len(sentence_tokens)):
#         global cnt
        sentence = sentence_tokens[i]
        if len(sentence)<5:
            pass
        news = sentence.replace('\n',' ')
        news = re.sub('[^\u0980-\u09FF]',' ',str(news)) #removing unnecessary punctuation
        
        text = text + news + " <EOS> "
          
        if ((i+1)% n_split)==0 and i!=0:
            cnt=cnt+1
            text = text + " <EOP> "
        if ((i+1)% n_split)!=0 and ((i+1)==len(sentence_tokens)):
            cnt = cnt+1
            text = text + " <EOP> "
    if cnt<=2:
        text=""
    return text

In [84]:
cleaning_headline(df.title[5]),df.title[5]

('অ্যান্ড্রয়েডের বড় ত্রুটির খোঁজ', 'অ্যান্ড্রয়েডের বড় ত্রুটির খোঁজ')

In [85]:
cleaning_documents(df.content[5]),df.content[5]

('গুগলের মোবাইল অপারেটিং সিস্টেম অ্যান্ড্রয়েডের বড় ধরনের ত্রুটির খোঁজ পেয়েছেন বলে দাবি করেছেন যুক্তরাষ্ট্রের প্রযুক্তি নিরাপত্তা গবেষণা প্রতিষ্ঠান ব্লুবক্সের গবেষকেরা  <EOS> গবেষকেরা বলছেন  অ্যান্ড্রয়েডের দুর্বলতা বা বাগ হিসেবে একটি  মাস্টার কী  এর খোঁজ পেয়েছেন তাঁরা  <EOS> এ সফটওয়্যার ত্রুটির কারণে সাইবার অপরাধীরা অ্যান্ড্রয়েড স্মার্টফোন থেকে তথ্য চুরি করতে পারে  <EOS>  <EOP> এক খবরে এ তথ্য জানিয়েছে বিবিসি অনলাইন  <EOS> গবেষকেদের দাবি  অ্যান্ড্রয়েড সফটওয়্যারের এই ত্রুটি ২০০৯ সালের পর থেকে উন্মুক্ত সব সংস্করণেই রয়েছে  <EOS> ক্রিপটোগ্রাফিক ভেরিফিকেশন প্রক্রিয়ার দুর্বলতার কারণেই অ্যান্ড্রয়েডে অতিরিক্ত ম্যালওয়্যারের আক্রমণ দেখা যায়  <EOS>  <EOP> ক্রিপটোগ্রাফিক ভেরিফিকেশন প্রক্রিয়ার মাধ্যমে বিভিন্ন অ্যাপ্লিকেশন পরীক্ষা করে দেখা হয়  <EOS> চলতি বছরের আগস্ট মাসে অনুষ্ঠিতব্য ব্ল্যাক হ্যাক হ্যাকারস সম্মেলনে অ্যান্ড্রয়েডের এ ত্রুটির বিস্তারিত জানানোর কথা জানিয়েছেন গবেষকেরা  <EOS> অ্যান্ড্রয়েডের সফটওয়্যার ত্রুটির বিষয়ে গুগল কর্তৃপক্ষ আনুষ্ঠানিকভাবে কোনো মন্তব্য করেনি  <EOS> 

In [86]:
df['content'] = df.apply(lambda x: cleaning_documents(x['content']), axis=1)
df['title'] = df.apply(lambda x: cleaning_headline(x['title']), axis=1)

In [87]:
print(len(df))
df = df.dropna()
print(len(df))

408471
408471


In [88]:
print(len(df))
df = df.drop(df[df['content']==""].index)
df = df.drop(df[df['title']==""].index)
print(len(df))

408471
277282


In [89]:
df.content[6],df.title[6]

('যুক্তরাষ্ট্রের বাজারে জিএসপি সুবিধা বন্ধ হওয়া প্রসঙ্গে স্বরাষ্ট্রমন্ত্রী মহীউদ্দীন খান আলমগীর বলেছেন  জিএসপি সুবিধা বন্ধ হওয়াতে বাংলাদেশের তেমন কোনো ক্ষতি হবে না  <EOS> আজ বৃহস্পতিবার বিকেলে চাঁদপুরের মতলব দক্ষিণ উপজেলায় নবগঠিত নারায়ণপুর পৌরসভার কার্যক্রম উদ্বোধন করতে গিয়ে প্রধান অতিথির বক্তব্যে মন্ত্রী এ কথা বলেন  <EOS>  ওয়াশিংটন টাইমস  এ বিরোধীদলীয় নেতা ও বিএনপির চেয়ারপারসন খালেদা জিয়ার লেখা নিবন্ধ প্রসঙ্গে স্বরাষ্ট্রমন্ত্রী বলেন   বাংলাদেশ সম্পর্কে তিনি  দুষ্টু কথা বলেছেন  <EOS>  <EOP> কোনো নেত্রী যদি দেশের উন্নয়ন বা মঙ্গল চান  তাহলে এ ধরনের কথা বলতে পারেন না  <EOS> নিন্দাসূচক কথা বলে তিনি দেশের অমঙ্গল করেছেন   কিশোরগঞ্জে আওয়ামী লীগ প্রার্থীর বিজয় প্রসঙ্গে মন্ত্রী বলেন  নৌকা প্রতীকে ভোট হলেই আওয়ামী লীগের প্রার্থী বিপুল ভোটে নির্বাচিত হন  <EOS> কিশোরগঞ্জের উপনির্বাচনই তার প্রমাণ  <EOS>  <EOP> তিনি বলেন   আমরা নির্বাচনে হস্তক্ষেপ করিনি  তাই চার সিটি করপোরেশন নির্বাচনে গণতন্ত্রের বিজয় হয়েছে  <EOS> কাজেই চার সিটি নির্বাচনে আমাদের যে পরাজয়ের কথা বলা হয়েছে  তাতে আমরা পর

In [90]:
# mask = (df['title'].str.len() <= 50) 
# df = df.loc[mask]
# df.head(2)

In [91]:
df.head(2)

,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content,id
0,গাজীপুর প্রতিনিধি,bangladesh,বাংলাদেশ,"০৪ জুলাই ২০১৩, ২৩:২৬","০৪ জুলাই ২০১৩, ২৩:২৭",[গাজীপুর],0,কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ বিক...,http://www.prothom-alo.com/bangladesh/article/...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,0
1,অনলাইন ডেস্ক,sports,খেলা,"০৪ জুলাই ২০১৩, ২৩:০৯","০৪ জুলাই ২০১৩, ২৩:১১",[টেনিস],0,সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি,http://www.prothom-alo.com/sports/article/19028,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...,1


### Splitting data equally for congruent and incongruent data generation
#### few sample data for generating incongruent data

In [92]:
congruent = df.iloc[:120000,:].reset_index()
incongruent1 = df.iloc[120000:150000,:].reset_index()
incongruent2 = df.iloc[150000:180000,:].reset_index()
incongruent3 = df.iloc[180000:210000,:].reset_index()

incongruent4 = df.iloc[210000:240000,:].reset_index()
# incongruent5 = df.iloc[300000:330000,:].reset_index()
# incongruent6 = df.iloc[330000:360000,:].reset_index()

sample = df.iloc[240000:,:].reset_index()

In [93]:
len(congruent),len(incongruent1),len(incongruent2),len(incongruent3),len(incongruent4),len(sample)

(120000, 30000, 30000, 30000, 30000, 37282)

## incongruent data genaration randomly mixing paragraph from sample data

In [95]:
def paragramp_seperation(content):
    content = content.split("<EOP>")
    content = [item for item in content if len(item)>4]
    
    x = []
    for line in content:
        no_space = line.strip()
        x.insert(0, no_space)
#     print(x)
    return x
    

In [96]:
def new_incongruent_preparation(content):
    content = paragramp_seperation(content)
#     print(len(content))
#     print(content)
    sample_content = sample['content'].sample(len(content))
#     print(sample_content)
    sample_content = sample_content.tolist() 
#     print(sample_content[0])
    sample_content = ''.join(sample_content)
    sample_para = paragramp_seperation(sample_content)
    
    n_len = len(content)
    try:
        n_replace = random.choice([i for i in range(1,n_len+1)])
    except:
        print(content)
        print(n_len)
    try:    
        sample_para_index = random.sample(range(0, len(sample_para)), n_replace)
        content_para_index = random.sample(range(0, len(content)), n_replace)
    except:
        print("2nd...")
        print(n_replace)
        print(content)
        print(sample_para)
        
    for i in range(0,n_replace):
        s,c = sample_para_index[i],content_para_index[i]
        content[c] = sample_para[s]
        
    content = [item+" <EOP> " for item in content]
    content = ' '.join([str(elem) for elem in content])
    k = random.choice([0,1,2,3])
    if len(content_para_index)==0:
        label=0
    else:
        label=1
    return content,label,len(content_para_index),content_para_index,k

In [97]:
new_incongruent_preparation(incongruent1.content[7]),incongruent1.content[7],incongruent1.title[7]

(('জাতীয় নাগরিকপঞ্জি ও নাগরিকত্ব বিল নিয়ে আসামসহ উত্তর পূর্বাঞ্চলে বিজেপি কিছুটা ব্যাকফুটে  <EOS> পশ্চিমবঙ্গে বিজেপির উপস্থিতি নিঃসন্দেহে জোরালো  <EOS> নিশ্চিতভাবেই তারা ভোট পাবে বেশি  <EOS> কিন্তু ভোটের দিন বুথে বুথে তৃণমূল কংগ্রেসের সঙ্গে মোকাবিলা করার মতো সাংগঠনিক শক্তি কি বিজেপি আয়ত্ত করেছে  <EOS> এটাও বড় প্রশ্ন বিজেপি ভোট লড়ছে পুরোপুরি নরেন্দ্র মোদির নামে  <EOS> <EOP>  সহকারী পুলিশ সুপার  মো  <EOS> সামছুজ্জামান বলেন  ওসির বিরুদ্ধে লাঠিপেটার অভিযোগটি সত্য নয়  <EOS> অবরোধকারীরা পুলিশকে দেখেই উত্তেজিত হয়ে পড়ে  <EOS> <EOP>  ঢাকার বাড্ডা এলাকায় আধিপত্য বিস্তার  অটোরিকশা স্ট্যান্ড ও ডিশ ব্যবসায়ীদের কাছে চাঁদার ভাগ বাঁটোয়ারা নিয়ে বিরোধের জেরে খুন হন বাড্ডা আওয়ামী লীগের সাধারণ সম্পাদক মো  <EOS> ফরহাদ আলী  <EOS> তাঁর খুনের নির্দেশ দেন বিদেশে থাকা শীর্ষ সন্ত্রাসীরা আজ শনিবার সকালে ডিএমপি মিডিয়া সেন্টারে প্রেস ব্রিফিংয়ে এসব কথা জানান অতিরিক্ত পুলিশ কমিশনার মো  <EOS> আবদুল বাতেন ১৫ জুন জুমার নামাজের পর বাড্ডার পূর্বাঞ্চল ১ নম্বর লেনের বায়তুস সালাম জামে মসজিদের পাশে ফরহাদ আলীকে প্রকাশ্যে গ

In [98]:
incongruent1['content'],incongruent1['label'],incongruent1['fake_para_len'],incongruent1['fake_para_index'],incongruent1['fake_type'] = zip(*incongruent1['content'].map(new_incongruent_preparation))
new_incongruent1 = incongruent1[[ 'id', 'title', 'content', 'label', 'fake_para_len', 'fake_para_index', 'fake_type']]

/opt/conda/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [99]:
incongruent2['content'],incongruent2['label'],incongruent2['fake_para_len'],incongruent2['fake_para_index'],incongruent2['fake_type'] = zip(*incongruent2['content'].map(new_incongruent_preparation))
new_incongruent2 = incongruent2[[ 'id', 'title', 'content', 'label', 'fake_para_len', 'fake_para_index', 'fake_type']]

In [100]:
incongruent3['content'],incongruent3['label'],incongruent3['fake_para_len'],incongruent3['fake_para_index'],incongruent3['fake_type'] = zip(*incongruent3['content'].map(new_incongruent_preparation))
new_incongruent3 = incongruent3[[ 'id', 'title', 'content', 'label', 'fake_para_len', 'fake_para_index', 'fake_type']]

In [101]:
incongruent4['content'],incongruent4['label'],incongruent4['fake_para_len'],incongruent4['fake_para_index'],incongruent4['fake_type'] = zip(*incongruent4['content'].map(new_incongruent_preparation))
new_incongruent4 = incongruent4[[ 'id', 'title', 'content', 'label', 'fake_para_len', 'fake_para_index', 'fake_type']]

In [ ]:
# incongruent5['content'],incongruent5['label'],incongruent5['fake_para_len'],incongruent5['fake_para_index'],incongruent5['fake_type'] = zip(*incongruent5['content'].map(new_incongruent_preparation))
# new_incongruent5 = incongruent5[[ 'id', 'title', 'content', 'label', 'fake_para_len', 'fake_para_index', 'fake_type']]

In [ ]:
# incongruent6['content'],incongruent6['label'],incongruent6['fake_para_len'],incongruent6['fake_para_index'],incongruent6['fake_type'] = zip(*incongruent6['content'].map(new_incongruent_preparation))
# new_incongruent6 = incongruent6[[ 'id', 'title', 'content', 'label', 'fake_para_len', 'fake_para_index', 'fake_type']]

In [102]:
new_incongruent1.head(2)

,id,title,content,label,fake_para_len,fake_para_index,fake_type
0,196932,মানুষের সঙ্গে ভালো আচরণের নির্দেশ আইজিপির,মানি লন্ডারিং প্রতিরোধে জেলা পুলিশকে প্রশিক্ষ...,1,2,"[1, 3]",2
1,196933,গণহত্যার হিসাব অনুমানভিত্তিক হয় হানিফ,সেখানে এক দুজন হিসাব করে সংখ্যা নির্ধারণ করা হ...,1,2,"[2, 1]",2


In [103]:
new_incongruent2.head(2)

,id,title,content,label,fake_para_len,fake_para_index,fake_type
0,243434,সন্তানকে নিয়ে পুকুরে ঝাঁপ দিয়ে মায়ের আত্মহত্যা,সব মিলিয়ে নিউইয়র্ক টাইমসের তোলা প্রশ্ন ফেলে দে...,1,4,"[0, 1, 4, 3]",0
1,243436,শ্রেণিকক্ষের সিলিং ফ্যান খুলে পড়ে শিক্ষার্থী আহত,১৯৭০ সালে পাঞ্জাব বিশ্ববিদ্যালয় থেকে উর্দু বিষ...,1,2,"[0, 2]",3


In [104]:
congruent['label'] = congruent.apply(lambda x: 0,axis=1)
congruent['fake_para_len'] = congruent.apply(lambda x: 0,axis=1)
congruent['fake_para_index'] = congruent.apply(lambda x: [],axis=1)
congruent['fake_type'] = congruent.apply(lambda x: -1,axis=1)
congruent.head(2)

,index,author,category,category_bn,published_date,modification_date,tag,comment_count,title,url,content,id,label,fake_para_len,fake_para_index,fake_type
0,0,গাজীপুর প্রতিনিধি,bangladesh,বাংলাদেশ,"০৪ জুলাই ২০১৩, ২৩:২৬","০৪ জুলাই ২০১৩, ২৩:২৭",[গাজীপুর],0,কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ বিক...,http://www.prothom-alo.com/bangladesh/article/...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,0,0,0,[],-1
1,1,অনলাইন ডেস্ক,sports,খেলা,"০৪ জুলাই ২০১৩, ২৩:০৯","০৪ জুলাই ২০১৩, ২৩:১১",[টেনিস],0,সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি,http://www.prothom-alo.com/sports/article/19028,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...,1,0,0,[],-1


In [105]:
new_congruent = congruent[[ 'id', 'title', 'content', 'label', 'fake_para_len', 'fake_para_index', 'fake_type']]

In [106]:
new_congruent.head(2)

,id,title,content,label,fake_para_len,fake_para_index,fake_type
0,0,কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ বিক...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,0,0,[],-1
1,1,সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...,0,0,[],-1


In [107]:
new_incongruent = pd.concat([new_incongruent1,new_incongruent2,new_incongruent3,new_incongruent4], axis=0)

In [108]:
print(len(new_incongruent))
new_incongruent = new_incongruent.dropna()
print(len(new_incongruent))

120000
120000


In [109]:
print(len(new_congruent))
new_congruent = new_congruent.dropna()
print(len(new_congruent))

120000
120000


In [110]:
len(new_incongruent),len(new_congruent)

(120000, 120000)

In [111]:
new_congruent.head()

,id,title,content,label,fake_para_len,fake_para_index,fake_type
0,0,কালিয়াকৈরে টিফিন খেয়ে ৫০০ শ্রমিক অসুস্থ বিক...,গাজীপুরের কালিয়াকৈর উপজেলার তেলিরচালা এলাকায়...,0,0,[],-1
1,1,সেমিফাইনাল বাধাও পেরিয়ে গেলেন লিসিকি,এবারের উইম্বলডনটা স্মরণীয় করে রাখার মিশনেই যে...,0,0,[],-1
2,2,সংসদে খালেদার অভিযোগের জবাব দিয়েছে ভারত,জাতীয় সংসদে বিএনপি চেয়ারপারসন ও বিরোধীদলীয় ...,0,0,[],-1
3,3,পাসওয়ার্ড ভুলে যান,সহজ পাসওয়ার্ডের কারণে অনলাইন অ্যাকাউন্ট সহজেই...,0,0,[],-1
4,4,চলে গেলেন মাউস উদ্ভাবক,কম্পিউটার মাউসের উদ্ভাবক ডগলাস অ্যাঙ্গেলবার্ট ...,0,0,[],-1


In [112]:
new_incongruent.head()

,id,title,content,label,fake_para_len,fake_para_index,fake_type
0,196932,মানুষের সঙ্গে ভালো আচরণের নির্দেশ আইজিপির,মানি লন্ডারিং প্রতিরোধে জেলা পুলিশকে প্রশিক্ষ...,1,2,"[1, 3]",2
1,196933,গণহত্যার হিসাব অনুমানভিত্তিক হয় হানিফ,সেখানে এক দুজন হিসাব করে সংখ্যা নির্ধারণ করা হ...,1,2,"[2, 1]",2
2,196934,গাজীপুরে ট্রেন বিকল আড়াই ঘণ্টা চলাচল বন্ধ,খবর পেয়ে জয়দেবপুর থেকে রেল জংশনের লোকজন ঘটনাস...,1,2,"[2, 1]",3
3,196935,নতুন শিক্ষানীতির দাবিতে সোচ্চার হওয়ার আহ্বান,সে কারণে তাঁরা বিরোধী দলে আছেন <EOS> আর বিএনপ...,1,4,"[1, 3, 0, 2]",2
4,196936,দলীয় তপস্যায় মগ্ন পিনাক,এতে পিনাকের রানের খিদেটা আরও বেড়েছে <EOS> সাম...,1,4,"[1, 3, 7, 6]",0


In [113]:
from sklearn.model_selection import train_test_split
train_congruent, test_val_train_congruent = train_test_split(new_congruent, test_size=0.05)
test_congruent, dev_congruent = train_test_split(test_val_train_congruent, test_size=0.5)
len(train_congruent),len(test_val_train_congruent),len(test_congruent),len(dev_congruent)

(114000, 6000, 3000, 3000)

In [114]:
from sklearn.model_selection import train_test_split
train_incongruent, test_val_train_incongruent = train_test_split(new_incongruent, test_size=0.05)
test_incongruent, dev_incongruent = train_test_split(test_val_train_incongruent, test_size=0.5)
len(train_incongruent),len(test_val_train_incongruent),len(test_incongruent),len(dev_incongruent)

(114000, 6000, 3000, 3000)

In [115]:
train = pd.concat([train_incongruent, train_congruent], axis=0)

In [116]:
test = pd.concat([test_incongruent, test_congruent], axis=0)

In [117]:
dev = pd.concat([dev_incongruent, dev_congruent], axis=0)

In [119]:
dev.head(2)

,id,title,content,label,fake_para_len,fake_para_index,fake_type
4925,296825,ফাঁসি মওকুফের পর রেয়াতও পেলেন যুবলীগ নেতা আসলাম,এর বিরুদ্ধে আপিল করা হলে সুপ্রিম কোর্ট আসলামের...,1,4,"[3, 6, 1, 7]",2
17546,224070,সাতক্ষীরায় সংঘর্ষে শ্রমিকনেতা নিহত,চাঁপাইনবাবগঞ্জে একটি হত্যা মামলায় দুজনকে যাবজ্...,1,3,"[2, 0, 1]",0


228000
133595


In [ ]:
test.head(2)

In [122]:
len(train),len(test),len(dev)

(228000, 6000, 6000)

In [125]:
train = train.sample(frac=1)
test = test.sample(frac=1)
dev = dev.sample(frac=1)

In [126]:
dev

,id,title,content,label,fake_para_len,fake_para_index,fake_type
16576,25808,বেকসুর খালাস পেলেন যাঁরা,পিলখানা হত্যা মামলায় অভিযোগ প্রমাণিত না হওয়া...,0,0,[],-1
13192,346146,ভল্ট থেকে বের করা ৪৫ লাখ টাকা হাওয়া,দাবি করা টাকা উদ্ধার ও জড়িত ব্যক্তিদের ধরতে পু...,1,1,[2],1
77181,128597,আওয়ামী লীগ কি ভয় পেয়েছে,যাঁরা সরকারের অন্ধ সমালোচক তার ভালো কাজও বাঁ...,0,0,[],-1
63457,104545,যুদ্ধ ও পররাষ্ট্রনীতির কৌশল বদলে দেবে থ্রিডি প...,ত্রিমাত্রিক প্রিন্টার দিয়ে অবিশ্বাস্য সব নকশ...,0,0,[],-1
17469,351132,স্ত্রীর মামলায় সেই এসআই গ্রেপ্তার,অন্যদিকে জার্মান সমর্থকেরাও আছেন খোশমেজাজে <E...,1,5,"[0, 3, 2, 4, 1]",1
...,...,...,...,...,...,...,...
10688,259762,৭০ লাখ টাকার জাল নোট উদ্ধার গ্রেপ্তার ১০,মরণাপন্ন অবস্থায় শেষবারের মতো হাসপাতালে যাওয়ার...,1,3,"[2, 0, 4]",3
28300,44631,নির্বাচনের আগে প্রয়োজন জঙ্গিবাদ মুক্ত করা,তথ্যমন্ত্রী হাসানুল হক ইনু বলেছেন নির্বাচন নি...,0,0,[],-1
22918,321496,পাঁচ মিনিটের ঝড়ে সহস্রাধিক ঘরবাড়ি বিধ্বস্ত ...,যে মাঠ পেরোলেই চন্দ্রিমা উদ্যানের একরাশ সবুজ ...,1,6,"[3, 2, 5, 1, 0, 4]",3
82807,137918,জ্বর নিয়ে ভাবছেন না ভাবনা,যদিও চলচ্চিত্রে টানা কাজ করে অসুস্থ হয়ে পড়েছ...,0,0,[],-1


In [127]:
train.to_csv('train.tsv',sep='\t',header=False,index=False)
test.to_csv('test.tsv',sep='\t',header=False,index=False)
dev.to_csv('dev.tsv',sep='\t',header=False,index=False)